In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [6]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [7]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    )
]

In [8]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [9]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [10]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection with Registry")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/05/18 22:46:05 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection with Registry' does not exist. Creating a new experiment.
2025/05/18 22:46:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/371217532274540904/runs/fadf41d6f5ef4cd080c1c77ea26f8529
🧪 View experiment at: http://localhost:5000/#/experiments/371217532274540904


2025/05/18 22:46:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/371217532274540904/runs/7df554bcba084931b148e13c366779eb
🧪 View experiment at: http://localhost:5000/#/experiments/371217532274540904


2025/05/18 22:46:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/371217532274540904/runs/27bf50bc565f465fa0842010be9c3a22
🧪 View experiment at: http://localhost:5000/#/experiments/371217532274540904


In [14]:
model = 'Random Forest'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Please type RunID 7df554bcba084931b148e13c366779eb


Registered model 'Random Forest' already exists. Creating a new version of this model...
2025/05/18 22:56:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Random Forest, version 2


🏃 View run Random Forest at: http://localhost:5000/#/experiments/371217532274540904/runs/7df554bcba084931b148e13c366779eb
🧪 View experiment at: http://localhost:5000/#/experiments/371217532274540904


Created version '2' of model 'Random Forest'.


In [18]:
model = 'Random Forest'
model_version = 2
model_uri = f"models:/{model}/{model_version}"
loaded_model = mlflow.sklearn.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [19]:
current_model_uri = f"models:/{model}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'anomaly-detection-prod'.
Copied version '2' of model 'Random Forest' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1747591486487, current_stage='None', description='', last_updated_timestamp=1747591486487, name='anomaly-detection-prod', run_id='7df554bcba084931b148e13c366779eb', run_link='', source='models:/Random Forest/2', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [20]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.sklearn.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])